In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# hack the paths, no package install needed
import sys
import os
repo_dir = '/media/user/repos/bulk-euclid-cutouts'
os.path.isdir(repo_dir)
sys.path.insert(0,repo_dir)

In [3]:
import logging
logging.basicConfig(level=logging.INFO)
import os

import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import pandas as pd

from bulk_euclid.utils import pipeline_utils
from bulk_euclid.external_targets import pipeline

INFO:matplotlib.font_manager:generated new fontManager
/tmp/ipykernel_336/3545559080.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Authenticate

In [9]:
# type in your COSMOS credentials
from astroquery.esa.euclid.core import Euclid
Euclid.login()

INFO:astroquery:Login to Euclid TAP server


INFO: Login to Euclid TAP server [astroquery.esa.euclid.core]


User:  mwalms01
Password:  ········


INFO:astroquery:OK
INFO:astroquery:Login to Euclid data service
INFO:astroquery:OK
INFO:astroquery:Login to Euclid cutout service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid data service [astroquery.esa.euclid.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid cutout service [astroquery.esa.euclid.core]


INFO:astroquery:OK


INFO: OK [astroquery.utils.tap.core]


### Set Pipeline Options and Load External Targets

In [5]:
# or you can use this config to run for real

cfg_dict = dict(
    base_dir='/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline',
    name='latest',
    release_name=None,  # allow tiles from any release
    bands=['VIS', 'NIR_Y', 'NIR_J'],  
    auxillary_products=['MERPSF','MERRMS'],  # ,'MERBKG'
    release_priority=['CALBLOCK_PV-005_R2', 'CALBLOCK_PV-005_R3', 'F-003_240321', 'F-003_240612' , 'F-006'],
    
    # ignore these
    download_dir=None,
    tile_dir=None,
    cutout_dir=None,
    fits_dir=None,
    sanity_dir=None
    
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc
pipeline.create_folders(cfg)  # fill in those missing paths


INFO:root:Saving to /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest


{'base_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline', 'name': 'latest', 'release_name': None, 'bands': ['VIS', 'NIR_Y', 'NIR_J'], 'auxillary_products': ['MERPSF', 'MERRMS'], 'release_priority': ['CALBLOCK_PV-005_R2', 'CALBLOCK_PV-005_R3', 'F-003_240321', 'F-003_240612', 'F-006'], 'download_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest', 'tile_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles', 'cutout_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/cutouts', 'fits_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/cutouts/fits', 'sanity_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/sanity'}

In [10]:

# TODO use your own csv, it just needs the columns specified here
# external_targets = pd.read_csv('/media/home/my_workspace/combined_karina_strong_lens_classifications.csv', usecols=['id_str', 'target_ra', 'target_dec'])
# external_targets['target_field_of_view'] = 20  # arcseconds

external_targets = pd.read_csv('/media/home/my_workspace/repos/bulk-euclid-cutouts/bulk_euclid/external_targets/LRGs_4sims_all.csv')
external_targets = external_targets.rename(columns={'ra': 'target_ra', 'dec': 'target_dec', 'ID': 'id_str'})
del external_targets['Unnamed: 0']
external_targets['target_field_of_view'] = 20  # arcseconds

In [12]:
len(external_targets)

5410

In [11]:
external_targets.nunique()

id_str                  5246
target_ra               5239
target_dec              5241
vdisp                   5407
z                       5410
target_field_of_view       1
dtype: int64

In [13]:
# TODO Karina, why are these duplicated?
external_targets = external_targets.drop_duplicates(subset=['id_str'], keep='first')

In [14]:
len(external_targets)

5246

In [15]:
assert not any(external_targets['id_str'].duplicated())

In [16]:
external_targets

,id_str,target_ra,target_dec,vdisp,z,target_field_of_view
0,EUC_15898_DESI-sel,268.870653,65.253093,238.68835,0.587760,20
1,EUC_2767_DESI-sel,150.341459,1.864402,212.48692,0.971600,20
2,EUC_2771_DESI-sel,150.364732,1.846961,273.08682,0.973256,20
3,EUC_21374_DESI-sel,267.736754,66.934327,189.43880,0.882198,20
4,EUC_17153_DESI-sel,272.923035,65.479365,278.98535,0.790216,20
...,...,...,...,...,...,...
5405,1237658609296933558,183.553030,68.557539,232.60570,0.540458,20
5406,1237658609296999171,183.020430,68.733176,383.99590,0.642441,20
5407,1237658609833870020,181.954280,68.700344,305.64170,0.520235,20
5408,1237671751363723635,180.609610,68.433166,231.08930,0.628734,20


### For each target, find a tile that covers it

In [17]:
targets_with_tiles = pipeline.get_matching_tiles(cfg, external_targets)
targets_with_tiles

INFO:astroquery:Query finished.
INFO:root:Found 2514 query results
INFO:root:Begin target/tile cross-match


INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Matched 5246 targets to 194 tiles
INFO:root:Targets with tile matches: 5177
/media/user/repos/bulk-euclid-cutouts/bulk_euclid/external_targets/pipeline.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets_with_tiles["tile_index"] = targets_with_tiles["tile_index"].astype(int)


,id_str,target_ra,target_dec,vdisp,z,target_field_of_view,tile_index,release_name
0,EUC_15898_DESI-sel,268.870653,65.253093,238.68835,0.587760,20,102158890,F-003_240612
1,EUC_2767_DESI-sel,150.341459,1.864402,212.48692,0.971600,20,101542818,CALBLOCK_PV-005_R2
2,EUC_2771_DESI-sel,150.364732,1.846961,273.08682,0.973256,20,101541378,CALBLOCK_PV-005_R2
3,EUC_21374_DESI-sel,267.736754,66.934327,189.43880,0.882198,20,102159774,F-003_240612
4,EUC_17153_DESI-sel,272.923035,65.479365,278.98535,0.790216,20,102158894,F-003_240612
...,...,...,...,...,...,...,...,...
5172,1237658609296933558,183.553030,68.557539,232.60570,0.540458,20,102160544,F-006
5173,1237658609296999171,183.020430,68.733176,383.99590,0.642441,20,102160810,F-006
5174,1237658609833870020,181.954280,68.700344,305.64170,0.520235,20,102160810,F-006
5175,1237671751363723635,180.609610,68.433166,231.08930,0.628734,20,102160541,F-006


In [18]:
targets_with_tiles['release_name'].value_counts()

release_name
F-003_240612          2758
F-006                 2004
CALBLOCK_PV-005_R2     363
F-003_240321            52
Name: count, dtype: int64

In [19]:
targets_with_tiles = targets_with_tiles.sort_values('tile_index') # [:1000]

The speed of the pipeline depends on the number of tiles to download. It takes a few minutes per tile.

In [20]:
targets_with_tiles['tile_index'].nunique(), len(targets_with_tiles)

(193, 5177)

### For every tile, download the flux and auxillary (PSF, RMS, BKG) data and make cutouts around each target in that tile

In [ ]:
pipeline.make_cutouts(cfg, targets_with_tiles)

INFO:root:Tile 101541377, 0 of 193
INFO:astroquery:Query finished.
INFO:root:Found 3 query results


INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Downloaded: {'VIS': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-VIS_TILE101541377-2E186_20240201T190014.111492Z_00.00.fits', 'MERPSF': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_CATALOG-PSF-VIS_TILE101541377-B2B5C3_20240201T190536.758003Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-VIS-RMS_TILE101541377-E3AEDE_20240201T185327.031980Z_00.00.fits'}, 'NIR_Y': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE101541377-6F8891_20240201T185319.156397Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-NIR-Y-RMS_TILE101541377-6B4880_202402

INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Downloaded: {'VIS': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-VIS_TILE101541378-737592_20240201T222127.194874Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-VIS-RMS_TILE101541378-65ED95_20240201T212225.072536Z_00.00.fits', 'MERPSF': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_CATALOG-PSF-VIS_TILE101541378-F7549B_20240201T223446.135532Z_00.00.fits'}, 'NIR_Y': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE101541378-143B40_20240201T213700.381755Z_00.00.fits', 'MERPSF': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_CATALOG-PSF-NIR-Y_TILE101541378-EB7826_2024

...

INFO:root:Downloaded: {'NIR_J': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-NIR-J_TILE101542819-A44F31_20240202T002853.274381Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-NIR-J-RMS_TILE101542819-714380_20240202T000622.807474Z_00.00.fits', 'MERPSF': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_CATALOG-PSF-NIR-J_TILE101542819-BF9AAD_20240202T010549.545547Z_00.00.fits'}, 'NIR_Y': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE101542819-3D9C48_20240202T002851.436958Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-NIR-Y-RMS_TILE101542819-1147

INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Downloaded: {'VIS': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-VIS_TILE101544255-6D25FA_20240202T071931.685336Z_00.00.fits', 'MERPSF': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_CATALOG-PSF-VIS_TILE101544255-F508E2_20240202T074724.664766Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-VIS-RMS_TILE101544255-FA1157_20240202T024030.071285Z_00.00.fits'}, 'NIR_Y': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE101544255-81A14_20240202T035059.082748Z_00.00.fits', 'MERPSF': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_CATALOG-PSF-NIR-Y_TILE101544255-2E5292_20240

INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Downloaded: {'NIR_Y': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE101544256-8DD33D_20240203T001101.427300Z_00.00.fits', 'MERPSF': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_CATALOG-PSF-NIR-Y_TILE101544256-3C998C_20240203T033644.054612Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-NIR-Y-RMS_TILE101544256-7C198A_20240202T031607.226528Z_00.00.fits'}, 'NIR_J': {'FLUX': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-NIR-J_TILE101544256-1961F0_20240203T000939.989862Z_00.00.fits', 'MERRMS': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-NIR-J-RMS_TILE101544256-2AB3